# Gather HCAL histos

In [ ]:
import pandas as pd
import numpy as np

# import cmsdials
from cmsdials import Dials
from cmsdials.auth.bearer import Credentials
from cmsdials.filters import (
    FileIndexFilters,
    LumisectionHistogram1DFilters,
    LumisectionHistogram2DFilters,
    LumisectionFilters,
    RunFilters,
    MEFilters
)

In [ ]:
creds = Credentials.from_creds_file()

dials = Dials(creds,workspace="hcal") # this will load into a DQM workspace by default (i.e. Tracker,HCAL,ECAL,etc.)

In [ ]:
kwargs= dict(
    status="FINISHED",
    dataset__regex="ZeroBias/Run2024[A-Z]-PromptReco/*",
    dim = 2,
    page_size = 500
            )

In [ ]:
allMEs= dials.mes.list_all(MEFilters(**kwargs))
allMEs

In [ ]:
RunsDF = dials.run.list_all(
                    RunFilters(**kwargs),
                    enable_progress = True
).to_pandas()

ls_mask = RunsDF.loc[:,"ls_count"] > 1000
selected_runs = RunsDF.loc[ls_mask].reset_index()
del RunsDF

In [ ]:
selected_runs.loc[::7].info()

In [ ]:
testDF = dials.h2d.list(LumisectionHistogram2DFilters(run_number=383687)
                       ).to_pandas()
testDF

# Now downloading all the data and making the files

In [ ]:
selected_runs

In [ ]:
import os

finished = [file for file in os.listdir() if "run" in file]
for run in selected_runs.run_number:
    for fin in finished:
        if f"{run:_}" in fin:
            print(f"skipping {run:_}")
        
    
# os.path.exists("run-378_239.parquet")

In [ ]:
for run in selected_runs.run_number:
    skip=False
    
    for fin in finished:
        if f"{run:_}" in fin:
            print(f"skipping {run:_}")
            skip=True
    
    if not skip:
        print(f"Starting download of {run:_}")
        irun_df = dials.h2d.list_all(
                                    LumisectionHistogram2DFilters(run_number=run,page_size=500),
                                     enable_progress=True
        ).to_pandas()
        
        irun_df.to_parquet(f"run-{run:_}.parquet")
        print(f"Done with {run = :,}.") 
        del irun_df